In [1]:
import numpy as np
import pandas as pd
import torch

In [14]:
class LogisticTorch:
    def __init__(self, lr=0.01, epochs=1000,epsilon = 1e-6):
        self.lr = lr
        self.epochs = epochs
        self.epsilon = epsilon
        self.theta = None
        self.loss_history = list()
        

    def fit(self, X, y):
        X = torch.tensor(X,dtype = torch.float32)
        y = torch.tensor(y,dtype = torch.float32).view(-1,1)
        ones = torch.ones((X.shape[0],1))
        X = torch.cat((ones,X), dim = 1)
        self.theta = torch.zeros((X.shape[1],1), dtype = torch.float32)

        for epoch in range(self.epochs):
            logits = X @ self.theta
            y_hat = torch.sigmoid(logits)
            grad = X.T @(y_hat - y)/len(y)
            oldtheta = self.theta.clone()
            loss = torch.mean((y_hat-y)**2)
            self.loss_history.append(loss)
            self.theta = self.theta - self.lr*grad

            if torch.norm(self.theta - oldtheta) < self.epsilon:
                print(f"Converged at epoch {epoch}")
                print(f"Final Loss in Model {loss.item():.5f}")
                break
        self.loss_history = torch.tensor(self.loss_history,dtype=torch.float32)

        

    def predict(self, X):
        X = torch.tensor(X, dtype = torch.float32)
        ones = torch.ones((X.shape[0],1),dtype = torch.float32)
        X = torch.cat((ones,X), dim =1 )
        logits = X @ self.theta
        return (logits>=0).float()

    def predict_proba(self, X):
        X = torch.tensor(X, dtype = torch.float32)
        ones = torch.ones((X.shape[0],1),dtype = torch.float32)
        X = torch.cat((ones,X), dim =1 )
        logits = X @ self.theta
        return torch.sigmoid(logits)


In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=200, noise=0.2, random_state=42)


In [33]:
split_ratio = 0.8

train_size = int(len(X)*split_ratio)
X_train = X[:train_size]
X_test = X[train_size:]

X_train = torch.tensor(X[:train_size], dtype = torch.float32)
y_train = torch.tensor(y[:train_size], dtype = torch.float32).view(-1,1)

X_test = torch.tensor(X[train_size:], dtype = torch.float32)
y_test= torch.tensor(y[train_size:], dtype = torch.float32).view(-1,1)

pd.DataFrame(X_train).head()

C:\Users\krish.HP.000\AppData\Local\Temp\ipykernel_12000\371791637.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y[:train_size], dtype = torch.float32).view(-1,1)
C:\Users\krish.HP.000\AppData\Local\Temp\ipykernel_12000\371791637.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test= torch.tensor(y[train_size:], dtype = torch.float32).view(-1,1)


,0,1
0,-1.106897,0.042293
1,0.956800,0.456750
2,0.733516,0.584617
3,1.111407,-0.309214
4,0.209082,0.000657


In [36]:
model = LogisticTorch()
model.fit(X_train,y_train)
y_hat_train = model.predict(X_train)
y_hat_train_probs = model.predict_proba(X_train)


C:\Users\krish.HP.000\AppData\Local\Temp\ipykernel_12000\2564597551.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X,dtype = torch.float32)
C:\Users\krish.HP.000\AppData\Local\Temp\ipykernel_12000\2564597551.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y,dtype = torch.float32).view(-1,1)
C:\Users\krish.HP.000\AppData\Local\Temp\ipykernel_12000\2564597551.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype = torch.float32)
C:\Users\krish.HP.000\AppData\Local\Temp\ip

In [ ]:
y_train = torch.tensor(y_train).view(-1,1)
train_accuracy = (y_hat_train == y_train).float().mean()
print(f"training accuracy: {train_accuracy:.4f}")


training accuracy: 0.8188


C:\Users\krish.HP.000\AppData\Local\Temp\ipykernel_12000\97020096.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).view(-1,1)


In [47]:
y_hat_test = model.predict(X_test)
test_accuracy = (y_hat_test == torch.tensor(y_test).view(-1,1)).float().mean()
print(f"test accuracy: {test_accuracy:.4f}")

print(y_hat_test.shape)
print(y_test.shape)


test accuracy: 0.8500
torch.Size([40, 1])
torch.Size([40, 1])


C:\Users\krish.HP.000\AppData\Local\Temp\ipykernel_12000\2564597551.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype = torch.float32)
C:\Users\krish.HP.000\AppData\Local\Temp\ipykernel_12000\1539583911.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_accuracy = (y_hat_test == torch.tensor(y_test).view(-1,1)).float().mean()


In [48]:
true_positive = ((y_hat_test == 1) & (y_test == 1)).float().sum()
false_positive = ((y_hat_test == 1) & (y_test == 0)).float().sum()
true_negative = ((y_hat_test == 0) & (y_test == 0)).float().sum()
false_negative = ((y_hat_test == 0) & (y_test == 1) ).float().sum()

recall = true_positive/(true_positive+false_negative)
precision = true_positive/(true_positive+false_positive)

print(f"Recall: {recall:.3f}")
print(f"Precision: {precision:.3f}")

Recall: 0.818
Precision: 0.900
